# Music Recommendation using K-Means Clusering

In [1]:
#1.import libraries
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set()


In [2]:
data = pd.read_csv("music.csv")
data.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [3]:
len(data)

25569

In [4]:
#2.data preprocessing

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25569 entries, 0 to 25568
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      25569 non-null  float64
 1   artists           25569 non-null  object 
 2   danceability      25569 non-null  float64
 3   duration_ms       25569 non-null  int64  
 4   energy            25569 non-null  float64
 5   explicit          25569 non-null  int64  
 6   id                25569 non-null  object 
 7   instrumentalness  25569 non-null  float64
 8   key               25569 non-null  int64  
 9   liveness          25569 non-null  float64
 10  loudness          25569 non-null  float64
 11  mode              25569 non-null  int64  
 12  name              25569 non-null  object 
 13  popularity        25569 non-null  int64  
 14  release_date      25569 non-null  object 
 15  speechiness       25569 non-null  float64
 16  tempo             25569 non-null  float6

In [6]:
data.isnull().sum()

acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
release_date        0
speechiness         0
tempo               0
valence             0
year                0
dtype: int64

In [7]:
df = data.drop(columns=[ 'id','release_date','year','name', 'artists'])
df.corr()


,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
acousticness,1.000000,-0.205249,-0.133389,-0.745771,-0.260715,0.336283,-0.010677,-0.005469,-0.504102,0.049648,-0.573705,-0.071965,-0.173354,-0.132634
danceability,-0.205249,1.000000,-0.158758,0.157935,0.216589,-0.223332,0.018772,-0.104384,0.194854,-0.041901,0.118785,0.269767,0.018540,0.556767
duration_ms,-0.133389,-0.158758,1.000000,0.099098,-0.039290,0.048093,0.000168,0.044232,0.058019,-0.038588,0.125491,-0.109479,-0.020062,-0.208963
energy,-0.745771,0.157935,0.099098,1.000000,0.103260,-0.253737,0.001449,0.088999,0.769314,-0.021555,0.494302,-0.139897,0.247891,0.327431
explicit,-0.260715,0.216589,-0.039290,0.103260,1.000000,-0.145097,0.010501,0.042714,0.085804,-0.064362,0.119289,0.364696,0.000279,-0.024775
instrumentalness,0.336283,-0.223332,0.048093,-0.253737,-0.145097,1.000000,0.000081,-0.042581,-0.353350,-0.035960,-0.362931,-0.137896,-0.076834,-0.177100
key,-0.010677,0.018772,0.000168,0.001449,0.010501,0.000081,1.000000,0.005220,0.003872,-0.105747,-0.001496,0.010072,-0.006933,0.015485
liveness,-0.005469,-0.104384,0.044232,0.088999,0.042714,-0.042581,0.005220,1.000000,0.044031,0.001972,-0.111029,0.109205,-0.006273,0.002454
loudness,-0.504102,0.194854,0.058019,0.769314,0.085804,-0.353350,0.003872,0.044031,1.000000,0.011613,0.437608,-0.269641,0.209560,0.296380
mode,0.049648,-0.041901,-0.038588,-0.021555,-0.064362,-0.035960,-0.105747,0.001972,0.011613,1.000000,0.012849,-0.064123,0.023969,0.028523


In [8]:
##Data transformation
from sklearn.preprocessing import MinMaxScaler
datatypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
normalization = data.select_dtypes(include=datatypes)
for col in normalization.columns:
    MinMaxScaler(col)

In [9]:
#importing K- means from sklearn
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30)
features = kmeans.fit_predict(normalization)
data['features'] = features
MinMaxScaler(data['features'])

MinMaxScaler(feature_range=0        12
1        18
2         0
3         1
4         0
         ..
25564    12
25565    17
25566    11
25567    18
25568    18
Name: features, Length: 25569, dtype: int32)

In [10]:
class Music_Recommendation():
    def __init__(self, dataset):
        self.dataset = dataset
    def recommend(self, songs, amount=1):
        distance = []
        song = self.dataset[(self.dataset.name.str.lower() == songs.lower())].head(1).values[0]
        rec = self.dataset[self.dataset.name.str.lower() != songs.lower()]
        for songs in tqdm(rec.values):
            d = 0
            for col in np.arange(len(rec.columns)):
                if not col in [1,6,12,14,18]:
                    d = d + np.absolute(float(song[col]) - float(songs[col]))
            distance.append(d)
        rec['distance'] = distance
        rec = rec.sort_values('distance')
        columns = ['artists', 'name']
        return rec[columns][:amount]

recommendations = Music_Recommendation(data)


In [11]:
recommendations.recommend("Lovers Rock", 5)

100%|██████████| 25568/25568 [00:01<00:00, 19821.43it/s]
C:\Users\PRASAD~1\AppData\Local\Temp/ipykernel_19196/1750719924.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec['distance'] = distance


,artists,name
16385,['O-Zone'],Dragostea Din Tei
11168,['Bob Marley & The Wailers'],Positive Vibration
7119,['Ella Fitzgerald'],Ev'ry Time We Say Goodbye
16590,['Linkin Park'],Somewhere I Belong
12287,['The Psychedelic Furs'],Love My Way


In [12]:
recommendations.recommend("Lovers Rock", 10)

100%|██████████| 25568/25568 [00:01<00:00, 20529.82it/s]
C:\Users\PRASAD~1\AppData\Local\Temp/ipykernel_19196/1750719924.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec['distance'] = distance


,artists,name
16385,['O-Zone'],Dragostea Din Tei
11168,['Bob Marley & The Wailers'],Positive Vibration
7119,['Ella Fitzgerald'],Ev'ry Time We Say Goodbye
16590,['Linkin Park'],Somewhere I Belong
12287,['The Psychedelic Furs'],Love My Way
15532,['Tracy Byrd'],I'm From The Country - Single Version
11921,['Robbie Dupree'],Steal Away - Remastered
1413,['Jimmie Rodgers'],Memphis Yodel
6940,['Frank Sinatra'],This Love Of Mine - 1998 Remastered
14588,['Los Tigres Del Norte'],Pacas De A Kilo


In [13]:
recommendations.recommend("Lovers Rock", 15)

100%|██████████| 25568/25568 [00:01<00:00, 23565.62it/s]
C:\Users\PRASAD~1\AppData\Local\Temp/ipykernel_19196/1750719924.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec['distance'] = distance


,artists,name
16385,['O-Zone'],Dragostea Din Tei
11168,['Bob Marley & The Wailers'],Positive Vibration
7119,['Ella Fitzgerald'],Ev'ry Time We Say Goodbye
16590,['Linkin Park'],Somewhere I Belong
12287,['The Psychedelic Furs'],Love My Way
15532,['Tracy Byrd'],I'm From The Country - Single Version
11921,['Robbie Dupree'],Steal Away - Remastered
1413,['Jimmie Rodgers'],Memphis Yodel
6940,['Frank Sinatra'],This Love Of Mine - 1998 Remastered
14588,['Los Tigres Del Norte'],Pacas De A Kilo
